In [13]:
import os
import cv2
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy.stats import skew, kurtosis
from skimage.feature import canny
from skimage.measure import shannon_entropy

In [14]:

# Load image in grayscale 

def load_gray(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    return img

In [15]:
# Extract features from each image 

def extract_features(img, file_path, class_label, pixel_density):
    # Basic info
    height, width = img.shape
    aspect_ratio = width / height
    file_size_kb = os.path.getsize(file_path) / 1024

    # Intensity stats
    mean_intensity = np.mean(img)
    std_intensity = np.std(img)
    skewness = skew(img.flatten())
    kurt = kurtosis(img.flatten())

    # Entropy and edge density
    entropy = shannon_entropy(img)
    edges = canny(img)
    edge_density = np.sum(edges) / (width * height)

    # Return as list in same order as DataFrame columns
    return [
        os.path.basename(file_path), class_label, pixel_density,
        width, height, aspect_ratio, file_size_kb,
        mean_intensity, std_intensity, skewness,
        kurt, entropy, edge_density
    ]


In [16]:
# Main function: Build metadata 

def build_metadata(root_folder):
    columns = [
        "file_name", "class_label", "pixel_density", "width", "height",
        "aspect_ratio", "file_size_kb", "mean_intensity", "std_intensity",
        "skewness", "kurtosis", "entropy", "edge_density"
    ]

    all_features = []

    for class_folder in os.listdir(root_folder):
        class_path = os.path.join(root_folder, class_folder)
        if not os.path.isdir(class_path):
            continue

        for pixel_folder in os.listdir(class_path):
            pixel_path = os.path.join(class_path, pixel_folder)
            if not os.path.isdir(pixel_path):
                continue

            for file_name in tqdm(os.listdir(pixel_path), desc=f"{class_folder}/{pixel_folder}"):
                if file_name.lower().endswith(".tif"):
                    file_path = os.path.join(pixel_path, file_name)
                    img = load_gray(file_path)
                    features = extract_features(img, file_path, class_folder, pixel_folder)
                    all_features.append(features)

    metadata = pd.DataFrame(all_features, columns=columns)
    return metadata

In [24]:
# Run the function 

#root_folder = r"C:\Users\Rishabh\OneDrive\Desktop\Dataset\Canon120-1-20251104T122320Z-1-001.zip"

#metadata_df = build_metadata(root_folder)

In [18]:
# Display Output 

print("\n Metadata table generated successfully!\n")
print(metadata_df.head())  # show first few rows as a table


 Metadata table generated successfully!

Empty DataFrame
Columns: [file_name, class_label, pixel_density, width, height, aspect_ratio, file_size_kb, mean_intensity, std_intensity, skewness, kurtosis, entropy, edge_density]
Index: []


In [ ]:
import os
import zipfile
import pandas as pd
from tqdm import tqdm

def extract_zip(zip_path):
    """Extracts a ZIP file if not already extracted, returns the folder path."""
    extract_to = zip_path.replace('.zip', '')
    if not os.path.exists(extract_to):
        print(f"Extracting: {zip_path}")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
    return extract_to


def build_metadata_for_all_zips(main_folder):
    """Scans all ZIP files in main_folder, extracts them, and builds metadata."""
    all_metadata = []

    # Loop through all files in main folder
    for file_name in os.listdir(main_folder):
        if file_name.lower().endswith(".zip"):
            zip_path = os.path.join(main_folder, file_name)
            
            # Extract ZIP and get extracted folder path
            extracted_folder = extract_zip(zip_path)
            
            # Build metadata for that extracted folder
            metadata_df = build_metadata(extracted_folder)
            all_metadata.append(metadata_df)

    # Combine all metadata into one DataFrame
    if all_metadata:
        final_metadata = pd.concat(all_metadata, ignore_index=True)
        return final_metadata
    else:
        print("No ZIP files found.")
        return pd.DataFrame()
    
   


NameError: name 'main_folder' is not defined

In [23]:
main_folder = r"C:\Users\Rishabh\OneDrive\Desktop\Dataset"
metadata_df = build_metadata_for_all_zips(main_folder)

metadata_df.to_csv("metadata_all.csv", index=False)
print("✅ Metadata created for all ZIPs and saved as 'metadata_all.csv'")


Extracting: C:\Users\Rishabh\OneDrive\Desktop\Dataset\Canon120-1-20251104T122320Z-1-001.zip


Canon120-1/300: 100%|██████████| 79/79 [02:19<00:00,  1.76s/it]


Extracting: C:\Users\Rishabh\OneDrive\Desktop\Dataset\Canon120-2-20251104T130247Z-1-001.zip


Canon120-2/300: 100%|██████████| 66/66 [01:56<00:00,  1.77s/it]


Extracting: C:\Users\Rishabh\OneDrive\Desktop\Dataset\Canon120-2-20251104T130247Z-1-002.zip


...

Extracting: C:\Users\Rishabh\OneDrive\Desktop\Dataset\HP-20251104T141025Z-1-001.zip


HP/300: 100%|██████████| 71/71 [01:41<00:00,  1.42s/it]


Extracting: C:\Users\Rishabh\OneDrive\Desktop\Dataset\HP-20251104T141025Z-1-002.zip


HP/300: 100%|██████████| 29/29 [00:45<00:00,  1.56s/it]

✅ Metadata created for all ZIPs and saved as 'metadata_all.csv'
